# HW_2_Javier_Gilabert_Sabater
## Análisis Predictivo

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Read Data
df = pd.read_csv('StackOverflow.csv')
df

,month,nltk,spacy,stanford-nlp,python,r,numpy,scipy,matlab,machine-learning,...,Plato,Sympy,Flair,stanford-nlp.1,pyqt,Nolearn,Lasagne,OCR,Apache-spark-mlib,azure-virtual-machine
0,09-Jan,0,0,0,631,8,6,2,19,8,...,0,1,0,0,5,0,0,5,0,0
1,09-Feb,1,0,0,633,9,7,3,27,4,...,0,0,0,0,5,0,0,11,0,0
2,09-Mar,0,0,0,766,4,4,2,24,3,...,0,0,0,0,7,0,0,2,0,0
3,09-Apr,0,0,0,768,12,6,3,32,10,...,0,0,0,0,11,0,0,5,0,0
4,09-May,1,0,0,1003,2,7,2,42,7,...,0,0,0,0,10,0,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,19-Aug,57,76,24,20257,4486,1111,212,371,583,...,0,33,0,22,144,0,0,40,16,25
128,19-Sep,54,62,16,19207,4474,1020,135,362,538,...,0,39,0,15,147,0,4,49,28,16
129,19-Oct,73,79,27,23414,5138,1310,188,523,750,...,0,72,0,27,170,0,3,70,21,19
130,19-Nov,72,79,14,23602,4883,1297,199,479,918,...,0,58,2,14,182,0,1,73,16,19


## 1. - Analizar los datos de MATLAB, entre otros análisis, se debe graficar la serie, estudiar la estacionalidad y la estacionariedad.

In [10]:
df_matlab  = pd.DataFrame()
df_matlab['month'] = df['month']
df_matlab['matlab'] = df['matlab']
df_matlab

,month,matlab
0,09-Jan,19
1,09-Feb,27
2,09-Mar,24
3,09-Apr,32
4,09-May,42
...,...,...
127,19-Aug,371
128,19-Sep,362
129,19-Oct,523
130,19-Nov,479
